/home/a/anaconda3/envs/graph/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
import torch
import numpy as np


from data_loading import parse_data, make_binary


class Data:
    def __init__(self, x, y_train, y_test, edge_index) -> None:
        self.x = x.float()
        self.edge_index = edge_index.int()
        self.indices = indices
        self.y_train = y_train.float()
        self.y_test = y_test.float()
        
    def to(self, device):
        self.x = self.x.to(device)
        self.y_train = self.y_train.to(device)
        self.y_test = self.y_test.to(device)
        self.indices = self.indices.to(device)
        self.edge_index = self.edge_index.to(device)
        return self

dataset = 'cora'
x_, edge_index_, y_ = parse_data(dataset)
num_classes = 1
num_node_features = x_.shape[1]
indices, y_binary_train, y_binary_test = make_binary(y_, 3, 0.05)
data = Data(x_, y_binary_train, y_binary_test, edge_index_)

In [23]:
data.y_train.dtype

torch.float32

In [25]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

import matplotlib.pyplot as plt

from sklearn.metrics import f1_score 

from loss import PULoss



# name_data = 'Cora'
# dataset = Planetoid(root= '', name = name_data)
# dataset.transform = T.NormalizeFeatures()


# print(f"Number of Classes in {name_data}:", num_classes)
# print(f"Number of Node Features in {name_data}:", num_node_features)
 

nnpu = False

class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        
        # 2개의 GAT layer를 쌓을 것이다.
        # ( 2번째 : multi-head attention 사용 )
        self.conv1 = GATConv(num_node_features, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        #-------------------------------------#
        # x : (2708, 1433)
        # edge_index : (2, 10556)
        #-------------------------------------#
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        #-------------------------------------#
        # x : (2708, 64)
        # 64 : hid 차원(8) x head 개수 (8)
        #-------------------------------------#
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        #-------------------------------------#
        # x : (2708, 7)
        #-------------------------------------#
        return torch.sigmoid(x)        
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GAT().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  

model.train()
for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    # print(out[data.train_mask].squeeze(1).dtype, data.y[data.train_mask].dtype)
    criterion = nn.BCELoss()
    criterion = PULoss(nnpu=nnpu)
    loss = criterion(out[data.indices].squeeze(1), data.y_train[data.indices])
    
    if epoch%100 == 0:
        print(loss.item())
    
    loss.backward()
    optimizer.step()

model.eval()
output = model(data)

pred = torch.where(output < 0.5, torch.tensor(0, device=device), 
            torch.tensor(1, device=device))

f1 = f1_score(data.y_test[data.indices].cpu(), pred[data.indices].cpu())
print(f1)

0.6843011975288391
0.6833291053771973
0.6637474894523621
0.6561630368232727
0.6273201107978821
0.7994579945799457


In [7]:
dataset = Planetoid(root= '', name = 'Cora')
dataset.num_features

1433

In [4]:
a = torch.tensor([[1], [2], [0]])
torch.sigmoid(a)
 

tensor([[0.7311],
        [0.8808],
        [0.5000]])

In [10]:
target = torch.empty(3).random_(2)
target.dtype

torch.float32